The ductbank model is the equilibrium thermal model (i.e., thermal mass is not considered) described by the following:

$$
    Q   \xrightarrow{} (T_C) 
        \xrightarrow{U_I} (T_S) 
        \xrightarrow{U_A} (T_D)
        \xrightarrow{U_S} (T_G)
$$

where
- $Q$ is the thermal heat from the cable losses (input)
- $T_C$ is the temperature of the cable core
- $U_I$ is the conductance of the cable insulation
- $T_S$ is the temperature of the cable surface
- $U_A$ is the conductance of the duct air gap
- $T_D$ is the temperature of the duct inner surface
- $U_S$ is the conductance of the duct shell, core, and fill
- $T_G$ is the temperature of the ground (input)

The equilibrium temperatures are calculated as follows

$$
    T_D = T_G + { Q \over U_S }, \qquad
    T_S = T_D + { Q \over U_A }, \qquad \mathrm{and} \qquad
    T_C = T_S + { Q \over U_I }
$$

The conductance of the air gap is computed as follows:

1. The cross sectional area of air $A$ is the overall cross section of the duct  $A_0$ less the overall cross section of cables in the duct $A_C$, including insulation, i.e.,

$$
    A = A_0 - A_C
$$

2. The mean distance from cable to duct $D$ is one half the square root of the cross sectional area of air, i.e.,

$$
    D = { \sqrt{A} \over 2 }
$$

3. The conductance of the air gap is the conductivity of air at ambient conditions divided by the mean distance, i.e.,

$$
    U_A = { 0.026 \over D }
$$


The conductance of the duct shell is computed as follows:
1. The duct shell cross-sectional area is

$$
    A_D = 4 L \sqrt{A_0}
$$

where $L$ is the length of the duct.

2. The conductance of the shell is computed as follows

$$
    U_S = { A_D \over U_{shell}^{-1} + U_{core}^{-1} + U_{fill}^{-1} } 
$$

where $U_*$ is taken from the table below according to the material used.

For the above calculations the following R-values and U-values may be used according the material type:

| Material | U-value (W/K.m) |
| :------- | --------------- | 
| ___Shell___
| PVC              | 0.19 |
| ___Core___       |||
| Concrete         | 0.92 |
| ___Fill soil___      |||
| Damp soil        | 1.00 |
| Average soil     | 0.80 |
| Dry soil         | 0.60 |
| ___Cable insulation___ |||
| Rubber           | 0.10 |
| Polyethylene     | 0.46 |
| __Air__|||
| Standard conditions | 0.026 |


# Example

Consider a duct with 6 cables of length 1000 meters, each with a loss of 1 kW in a duct bank of nominal cross sectional area 0.1 m$^2$.  The cables are 2.5 cm in diameter with 1 cm of rubber insulation.  The duct shell is 1 cm of PVC. The duct core is 40 cm thick concrete and the fill is 60 cm of average moisture soil. The ground temperature is $10^{\mathrm{o}}\mathrm{C}$.

In [1]:
# conditions as specified
from math import *
N = 6 # number of cables
L = 1e3 # duct length (m)
Q = 1e3 # cable loss (W)
A0 = 0.01 # duct cross sectional area (m^2)
TG = 10.0 # ground temperature (degC)
DI = 0.01 # cable insulation thickness (m)
DC = 0.025 # cable diameter (M)
DS = 0.01 # shell thickness
DD = 0.40 # duct thickness
DF = 0.60 # fill thickness

In [2]:
# U-value table
U = {
    "PVC" : 0.19,
    "CONCRETE" : 0.92,
    "WETSOIL" : 1.00,
    "SOIL" : 0.80,
    "DRYSOIL" : 0.60,
    "RUBBER" : 0.10,
    "PE" : 0.46,
    "AIR" : 0.026,
} # W/K.m (per-unit thickness)

In [3]:
# insulator conductance = cable area * insulation U-value / insulation thickness
IA = N*(DC*pi) * L # insulation area (m^2)
UI = U["RUBBER"] / DI * IA # W/K

In [4]:
# air conductance
AC = N*(DC/4)**2*pi # cable cross sectional area (m^2)
A = A0-AC # air cross sectional area (m^2)
if A <= 0: raise Exception("duct is too small")
D = sqrt(A)/2 # mean air gap from cable to duct (m)
AD = 4*sqrt(A)*L # effective air surface area
UA = U["AIR"]/D*AD # W/K

In [5]:
# duct conductance
AD = 4*sqrt(A0)*L # duct surface area
Ushell = U["PVC"]/DS # PVC
Ucore = U["CONCRETE"]/DD # Concrete duct thickness
Ufill = U["SOIL"]/DF # soil thickness
US = AD/(1/Ushell+1/Ucore+1/Ufill) # W/K

In [6]:
# temperature gradient
TD = TG + N*Q/US
TS = TD + N*Q/UA
TC = TS + N*Q/UI
print('                       ','           '.
      join(['%2s=%-5.0f'%(x,globals()[x]) for x in ['US','UA','UI']]))
print('Q=%-5.0f --->'%(N*Q),' ------> '.
      join(["(%2s=%-5.1f)"%(x,globals()[x]) for x in ["TC","TS","TD","TG"]]))


                        US=323             UA=208             UI=4712 
Q=6000  ---> (TC=58.7 ) ------> (TS=57.4 ) ------> (TD=28.6 ) ------> (TG=10.0 )
